# Using a Keras Long Short-Term Memory (LSTM) Model to Predict Stock Prices

**_LSTM_**'s are very powerful in sequence prediction problems because they're able to store past information. This is important in our case because the previous price of a stock is crucial in predicting its future price.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set(style='darkgrid', palette='viridis')

## Load Dataset

In [3]:
data = pd.read_csv('./__lhlrepos__/stockprice/NSE-TATAGLOBAL.csv')
train = data.iloc[:, 1:2].values

In [4]:
data.head()

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.20,233.50,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.10,233.80,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.50,235.00,234.25,2240909,5248.60
3,2018-09-25,233.30,236.75,232.00,236.25,236.10,2349368,5503.90
4,2018-09-24,233.55,239.20,230.75,234.00,233.30,3423509,7999.55


The `Open` column is the starting price while the `Close` column is the final price of a stock on a particular trading day. The `Hihg` and `Low` columns represent the highest and lowest prices for a certain day.

## Feature Scaling

In [5]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(train)

## Creating Data with Timesteps

LSTM's expect our data to be in a specific format, usually a 3D-array. We start by creating data in 60 timesteps and converting it into an array using NumPy. Next, we convert the data into a 3D array with `xtrain` samples, 60 timestamps, and one feature at each step.

In [6]:
xtrain = []
ytrain = []

for i in range(60, 2035):
    xtrain.append(training_set_scaled[i-60:i, 0])
    ytrain.append(training_set_scaled[i, 0])

xtrain, ytrain = np.array(xtrain), np.array(ytrain)
xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], 1))

## Building the LSTM    

In order to build the LSTM, we need to import a couple of modules from `Keras`.
1. `Sequential` for initializing the neural network.
2. `Dense` for addigin a densely connected neural network layer.
3. `LSTM` for adding the Long Short-Term Memory layer
4. `Dropout` for adding dropout layers that prevent overfitting.

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

We add the LSTM layer and later add a few `Dropout` layers to prevent overfitting. Below are the parameters our LSTM layer will initialize with:
1. `50 units` which is the dimensionality of the output space.
2. `return_sequences=True` which determines whether to return the last output in the output sequene, or the full sequence.
3. `input_shape` as the shape of our training set.

When defining the `Dropout` layers, we specify `0.2` to indicate that we want 20% of the layers to be dropped at random. Thereafter, we add the `Dense` layer specifying the output of 1 unit. After this, we compile our model using the `adam` optimizer and set the loss as the `mean_squared_error`.

Next, we fit the model to run on `100 epochs` with a `batch size of 32`.

In [8]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (xtrain.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(xtrain, ytrain, epochs = 100, batch_size = 32)

2022-06-08 16:29:04.518329: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


: 

: 

## Predicting Future Stock using the Test Set

In [ ]:
datatest = pd.read_csv('./__lhlrepos__/stockprice/tatatest.csv')
real_stock_price = datatest.iloc[:, 1:2].values

In order to predict future stock prices, we need to do a couple of things after loading in the test set:
1. Merge the training set and the test set on the 0 axis.
2. Set the time step as 60.
3. Use `MinMaxScaler` to transform the new dataset.
4. Reshape the dataset as done previously.

Then after making the predictions, we use `inverse_transform` to get back the stock prices in normal readable format.

In [ ]:
# combine the datasets for when stock prices open for the following daily observation
dataset_total = pd.concat((data['Open'], datatest['Open']), axis = 0)

# define the inputs (previous time steps)
inputs = dataset_total[len(dataset_total) - len(datatest) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

# define the outputs
Xtest = []
for i in range(60, 76):
    Xtest.append(inputs[i-60:i, 0])
Xtest = np.array(Xtest)
Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], 1))

# make predictions
predicted_stock_price = regressor.predict(Xtest)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

## Plotting the results

In [ ]:
fig,ax = plt.subplots(figsize=(16,10))

ax.plot(real_stock_price, color = 'black', label = 'TATA Stock Price')
ax.plot(predicted_stock_price, color = 'green', label = 'Predicted TATA Stock Price')
ax.title('TATA Stock Price Prediction')
ax.xlabel('Time')
ax.ylabel('TATA Stock Price')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

From the plot, we can see that the real stock price went up while our model also predicted that the price of the stock will go up. This clearly shows how powerful LSTMs are for analyzing time series and sequential data.